
# Unsupervised Tumor Clustering via Expectation–Maximization (EM)

Background & Motivation: Distinguishing cancerous (malignant) tumors from benign tumors is a critical healthcare task. Typically this is a supervised classification problem, but here we consider an unsupervised scenario: suppose we have biological measurements of tumors without known labels. Can we discover the underlying grouping of tumors into “malignant” vs “benign” purely from the data? This is a realistic situation when labeled data is scarce or expensive to obtain. We will use a Gaussian Mixture Model (GMM) and the Expectation–Maximization (EM) algorithm to cluster tumors, illustrating how EM can estimate parameters of latent-class models. This mirrors real-world use of EM in clustering, anomaly detection, and incomplete-data problems (e.g. grouping patients by disease subtype from symptoms). It also demonstrates EM’s power to handle latent variables (in this case, the unknown tumor class).

## Wisconsin Diagnostic Breast Cancer (WDBC) dataset
This is a well-known public dataset containing 569 tumor samples (212 malignant, 357 benign) described by 30 continuous features derived from cell nuclei images. Each sample is labeled M (malignant) or B (benign), though our clustering algorithm will not use these labels for training – we’ll only use them afterward to evaluate clustering performance. The features include radius, texture, area, smoothness of the cell nuclei, etc. ￼. This rich feature set makes the clusters (malignant vs benign) somewhat separable in the high-dimensional space, which EM should capitalize on. The real-world relevance is high: these measurements come from actual biopsies, and an unsupervised model that automatically separates likely malignant from benign tumors could assist in preliminary screening or insight into the data structure.

## Problem Setup (GMM Formulation): 

We assume the data can be modeled as a mixture of two multivariate Gaussian distributions: one corresponding to benign tumors and one to malignant. Formally, let $x_i \in \mathbb{R}^{30}$ be the feature vector for the $i$-th tumor. We model the density as
$$
p(x_i \mid \Theta) = \pi_1 \mathcal{N}(x_i \mid \mu_1, \Sigma_1) + \pi_2 \mathcal{N}(x_i \mid \mu_2, \Sigma_2),
$$
where $\Theta={\pi_1,\pi_2,\mu_1,\mu_2,\Sigma_1,\Sigma_2}$ are the model parameters. Here $\pi_k$ is the mixing proportion (prior probability of cluster $k$, with $\pi_1+\pi_2=1$), $\mu_k$ is the mean feature vector for cluster $k$, and $\Sigma_k$ the $30\times30$ covariance matrix. Initially, these parameters are unknown. Our goal is to learn $\Theta$ that maximizes the likelihood of the observed data (i.e. fit the mixture to the data distribution). 

The log-likelihood is
$$
\ell(\Theta) = \sum_{i=1}^{n} \log\Big[\pi_1 \mathcal{N}(x_i;\mu_1,\Sigma_1) + \pi_2 \mathcal{N}(x_i;\mu_2,\Sigma_2)\Big].
$$
This is difficult to maximize directly because of the log-sum (the assignment of each $x_i$ to a component is unknown). However, if we introduce latent variables $z_i \in {1,2}$ indicating the cluster identity of each point, the problem would simplify (complete-data log-likelihood is easier). EM tackles this by iteratively estimating the expected latent assignments (E-step) and then updating parameters to maximize that expected complete-data log-likelihood (M-step).


In [2]:
from sklearn.datasets import load_breast_cancer
import pandas as pd

# Load data
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name="target")  # 0 = malignant, 1 = benign

print(X.shape, y.value_counts())  # Check dimensions and class balance
print(X.head())

(569, 30) target
1    357
0    212
Name: count, dtype: int64
   mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
0        17.99         10.38          122.80     1001.0          0.11840   
1        20.57         17.77          132.90     1326.0          0.08474   
2        19.69         21.25          130.00     1203.0          0.10960   
3        11.42         20.38           77.58      386.1          0.14250   
4        20.29         14.34          135.10     1297.0          0.10030   

   mean compactness  mean concavity  mean concave points  mean symmetry  \
0           0.27760          0.3001              0.14710         0.2419   
1           0.07864          0.0869              0.07017         0.1812   
2           0.15990          0.1974              0.12790         0.2069   
3           0.28390          0.2414              0.10520         0.2597   
4           0.13280          0.1980              0.10430         0.1809   

   mean fractal dimension  ... 

## Tasks:

- Lay out the details of using the EM algorithm to solve the GMM model.

- Implement it in python and perform clustering on the cancer data. Do not use existing functions, but write your own code for the algorithm.

- Evaluate the clustering result with the true labels.